This notebook will walk you through creating a tensorflow model for batch predictions with [Cloud ML Engine](https://cloud.google.com/ml-engine/docs/).

If you're new to Jupyter notebooks/Colab:  you can run  a block (a.k.a cell) of code by selecting it with your mouse and press SHIFT and ENTER at the same time to run it.

For more details, including how to view this file and connect it to a python interpreter check out [Welcome to Colab](https://colab.research.google.com/notebooks/welcome.ipynb).

# Getting batch predictions from a pretrained model with Cloud ML Engine

## Introduction
Medical images are scarce. For some rare diseases there may only be tens of cases that are available for imaging. Even when there are many patients, imaging can still be a burden to patients and a cost to healthcare systems.

To understand an image, our model will need general perception skills. It will need representations for lines and shapes at different levels of complexity. Because our medical data is so precious, it is best not to waste it on teaching our model such basic skills. Also, our model wouldn't get very good at perception if it only learned from a small set of images.

In this notebook, we're going to walk through reducing images into a language of lines and shapes using [Cloud ML Engine](https://cloud.google.com/ml-engine/docs/) and a model that has already learned the skills of perception from [the sea of images on the web](http://www.image-net.org/).

In [0]:
import tensorflow as tf
tf.keras.backend.set_learning_phase(0)

## The plan

Our goal is to convert each image in the dataset into a compact vector representation.
The image we start with has complicated correlations between its pixels that form blobs, lines and shadows.
This makes for some redundancy in the information each pixel tells us.

Neural networks attempt to sequentially remove this redundancy until only the information they care about is left. Often this information is a label, like `apple` or `cat`. Convolutional neural networks (CNN) are a particular species of neural networks that are designed to take advantage of the fact that moving an object around in an image doesn't change its identity.

We're going to use a CNN, called [Xception](https://arxiv.org/pdf/1610.02357.pdf), that has already been trained to recoginize things like apples and cats using data from [ImageNet](http://www.image-net.org/). As Xception sequentially reduces our image towards an extremely compact represention (such as the label `apple`), we're going to pull out somewhere in the middle. This will give us a summary of our image as a 2048 dimensional vector. Furthermore we expect the correlations between each of the elements (i.e. features) of this vector to be small.

In summary, we're going to accomplish two things
1. Supplement our dataset with all the images in ImageNet
2. Reduce the dimensionality of our data to avoid [overfitting](https://en.wikipedia.org/wiki/Overfitting) without ignoring the special stucture our data has since it's an image.

## Implementing the plan

Running data through state of the art neural networks can take a lot of computing horse power. Although we won't get into it in this notebook, training (i.e. optimizing the paramaters of) the network is even more demanding.

Fortunately, [Cloud ML Engine](https://cloud.google.com/ml-engine/docs/) can take care of balancing the work across an effectively unlimited number of GPUs or TPUs so we can get our results quickly.

In particular, we're going to use Cloud ML's [batch prediction feature](https://cloud.google.com/ml-engine/docs/tensorflow/batch-predict) to quickly get feature vectors for every image in our dataset.

### Loading the images

The first step in defining the model is to define how to load data. Cloud ML engine supports JSON and [TFRecord](https://www.tensorflow.org/guide/datasets#consuming_tfrecord_data) file formats for input into batch prediction.

We define functions to build inputs using both of these datatypes.

In [0]:
def example_serving_input_fn():
  """Creates tensors for TFExample input from TFRecord files.

  Returns:
    Tuple[Dict[str, tf.Tensor], Dict[str, tf.Tensor]]:
      The tensorflow tensors for the first and second stage of data
      ingestion respectively. In the first stage, data is loaded using
      standard Cloud ML input conventions for the TFRecord data type.
      In the second stage, the data from the first stage is converted
      in the form we have defined as input for our model.
  """
  feature_spec = {key: tf.FixedLenFeature(shape=[], dtype=tf.string)
                  for key in ('png_bytes', 'study', 'series', 'instance')}

  example_bytestring = tf.placeholder(tf.string, shape=[None])
  input_tensors = tf.parse_example(example_bytestring, feature_spec)
  return {'example': example_bytestring}, input_tensors

# Not used, but included to demonstrate how JSON input works
def json_serving_input_fn():
  """Creates tensors for JSON input.
  """
  # png_bytes is automatically decoded from base64 by Cloud ML Engine.
  # Its naming is important. See "Binary data in prediction input" in
  # https://cloud.google.com/ml-engine/docs/tensorflow/online-predict#formatting_instances_as_json_strings
  # for details.
  input_tensors = {key: tf.placeholder(tf.string, shape=[None], name=key)
                   for key in ('png_bytes', 'study', 'series', 'instance')}
  return input_tensors, input_tensors

serving_input_fn = example_serving_input_fn

simple_input_signature, input_tensors = serving_input_fn()
output_tensors = dict()
# Instance keys, see
# https://cloud.google.com/ml-engine/docs/tensorflow/prediction-overview#instance_keys
# for more information
for instance_key in ('study', 'series', 'instance'):
  output_tensors[instance_key] = tf.identity(input_tensors[instance_key])

### Formatting the images

The next step is converting binary png encoded image data into an array of pixel intensities.

Different neural networks require their data to be in different formats. In particular, the [version of Xception](https://keras.io/applications/#xception) that we're going to use requires its data to be a $299 \times 299 \times 3$ array of pixels with values between $-1$ and $+1$. The first two dimensions are the height and width of the image (i.e. its _resolution_), the last dimension is the three primary colors (red, green, blue) that compose each pixel.

In [0]:
def parse_png(png_bytes, height=299, width=299):
  """Decode a blob of png bytes and resize to fit CNN input size.

  Returns:
    tf.Tensor:
      A (H=height)(W=width)(C=3) array with pixel
      intensities in the range [-1, 1].
  """
  # Decode the png_bytes into a HW(C=3) array.
  u8image = tf.image.decode_png(png_bytes, channels=3)
  # Resized image pixel instansities are still in [0, 255], but are
  # promoted to float32's by the bilinear averaging.
  f32image_resized = tf.image.resize_images(u8image, (height, width))
  # Convert to data with pixel intensities in the range [-1, 1].
  tf_image = tf.keras.applications.xception.preprocess_input(f32image_resized)
  return tf_image

In [0]:
processed_images = tf.map_fn(parse_png,
                             input_tensors['png_bytes'],
                             dtype=tf.float32,
                             back_prop=False)

### Running the neural network

The last step in defining the model is defining how the CNN will run on the data. Here's a high level description of each of the arguments

* `include_top=False`:  _stop early! I don't want a label like apple, I want a vector summary of the image_
* `pooling=max`:  _do one more summarizing step so I get a 2048 dimensional vector_
* `weights=imagenet`:  _use a version of this model that has already been trained using images from ImageNet_
* `input_tensor=processed_images`:  _this is the tensor holding the input for the CNN_

In [0]:
fv_model = tf.keras.applications.xception.Xception(
    include_top=False,
    pooling='max',
    weights='imagenet',
    input_tensor=processed_images)

output_tensors['feature_vector'] = fv_model.output

Congratulations! Our model is built!
Let's save it to a directory so that we can upload it to Cloud ML engine.

In [0]:
tf.saved_model.simple_save(
     # the tensorflow session to save
    tf.keras.backend.get_session(),
    # path to the model, the 1 stands for the model version number by convention
    'xception_fv/1',
    inputs=simple_input_signature,
    outputs=output_tensors)

### Getting batch predictions with Cloud ML Engine


For the curious, we can inspect our model using the [saved_model_cli](https://www.tensorflow.org/guide/saved_model#cli_to_inspect_and_execute_savedmodel) tool that packaged with tensorflow.

You can also test out your model at the command line with this tool, which is useful for debugging.

In [0]:
%%bash
saved_model_cli show --dir xception_fv/1 --all

Upload the model to Google Cloud Storage (GCS) so that Cloud ML engine will be able to see it. Remeber to edit the below `gcs_model_path` paramater so that it points to one of your GCS buckets. You can do this by editing the code directely, or, if you're using this notebook from Colab, editting the parameter box on the right. Remeber to press SHIFT+ENTER to run it.

In [0]:
# Replace me!
gcs_model_path = 'gs://my_bucket/xception_fv/1' #@param

In [0]:
%%bash -s {gcs_model_path}
gsutil rsync -r xception_fv/1 $1

Next, we'll load the model into Cloud ML engine.
You can also do this using the [Cloud ML engine web interface](https://console.cloud.google.com/mlengine (select `Models` and then `+New Model`)

In [0]:
%%bash -s {gcs_model_path}
gcloud ml-engine models create xception_fv \
    --description=\
"Extract feature vectors from png encoded images served in a TFRecord "\
"using the Xception CNN trained on imagenet"

gcloud ml-engine versions create version_1 \
    --model=xception_fv \
    --origin=$1

What remains is to launch a batch prediction job to run our model all all the images in your dataset.

To create the input, you can use the provided Cloud Dataflow pipeline `png_to_tfrecord.py`.
TFExamples of the required for form are be built using code like this:

```
def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

example = tf.train.Example(
    features=tf.train.Features(feature={'study': _bytes_feature(study_uid),
                                        'series': _bytes_feature(series_uid),
                                        'instance': _bytes_feature(instance_uid),
                                        'png_bytes': _bytes_feature(png_bytes)}))
```

And these examples are then saved to a TFRecord file.


##### Aside: Using JSON input instead of TFRecord

If you want to rebuild this model so that it uses JSON input instead of TFRecord input
change `serving_input_fn = example_serving_input_fn` to `serving_input_fn = json_serving_input_fn` in this notebook, rebuild the model and supply your input in a text file with lines like this (use `--data-format=text`
for this)

```
{"study": study_uid, "series": series_uid, "instance": instance_uid, "png_bytes": {"b64": base64.b64encode(png_bytes)}}
```

Cloud ML Engine will automatically decode the base64 binary string, see the heading "Binary data in prediction input" in [this document](https://cloud.google.com/ml-engine/docs/tensorflow/online-predict#formatting_instances_as_json_strings) for details.
##### End aside

In [0]:
job_name = 'feature_vector_batch_predict' #@param
# Replace me!
region = 'us-central1' #@param
input_path = 'gs://my_bucket/input_tfrecords/**.tfrecord' #@param
output_path = 'gs://my_bucket/cloudml_output/feature_vectors.txt' #@param

In [0]:
%%bash -s {job_name} {input_path} {output_path} {region}
gcloud ml-engine jobs submit prediction $1 \
  --model=xception_fv \
  --input-paths=$2 \
  --output-path=$3  \
  --region=$4 \
  --data-format=tf-record

You can watch the progress of the job on the [Cloud ML dashboard](https://console.cloud.google.com/mlengine). When its done, you will find the output on Cloud Storage in the `${output_path}` you provided The format of the output will be a text file with a JSON entry on each line of the form

```
{"study": study_uid, "series": series_uid, "instance": instance_uid, "feature_vector": [1.61,-0.35,..]}
```

## What now?

Now that we have a feature vectors describing each of the images in our dataset, the challenges of working with images (high-dimensionality and complicated correlations) are mostly abated. We are free to use the feature vector as a representation of the image in any statistical analysis of our data.

Some options:

* Classification: Logistic regression, support vector machines, dense neural networks
* Visualization: t-SNE, PCA
* Clustering: k-means, PCA + guassian mixtures

No matter what you choose for the next step of your analysis, loading your data into BigQuery is a great next step. From BigQuery you can join your image feature vectors with DICOM metadata and electronic healthrecords. You can export your queries to a huge variety of formats, or train your models directly using [BigQuery ML](https://cloud.google.com/bigquery/docs/bigqueryml-intro). To this end, we've provided [load_feature_vectors.py](load_feature_vectors.py): a Cloud Dataflow pipeline for loading the text file of feature vectors given to you by Cloud ML engine into BigQuery.